# SEPTA Regional Rail – Weekday Service Performance (GTFS)

**Goal:** Build a weekday-only dataset from SEPTA’s static GTFS feed and export clean tables for a Power BI operations dashboard.

**Outputs (for Power BI):**
- `fact_trips_weekday.csv`
- `hourly_summary_weekday.csv`
- `route_summary_weekday.csv`

In [14]:
import os
import io
import zipfile
import requests
import pandas as pd

pd.set_option("display.max_columns", None)

In [15]:
GTFS_URL = "https://www3.septa.org/developer/gtfs_public.zip"
PROJECT_DIR = "septa_weekday_dashboard"
RAW_DIR = os.path.join(PROJECT_DIR, "raw")
RAIL_DIR = os.path.join(PROJECT_DIR, "rail")
EXPORT_DIR = os.path.join(PROJECT_DIR, "exports")

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(RAIL_DIR, exist_ok=True)
os.makedirs(EXPORT_DIR, exist_ok=True)

print("Downloading GTFS...")
resp = requests.get(GTFS_URL, timeout=60)
resp.raise_for_status()

print("Extracting main zip...")
with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    z.extractall(RAW_DIR)

print("Main GTFS extracted. Contents:")
print(os.listdir(RAW_DIR))

Extracting main zip...
Main GTFS extracted. Contents:
['google_bus.zip', 'google_rail.zip']


In [16]:
rail_zip_path = os.path.join(RAW_DIR, "google_rail.zip")

print("Extracting google_rail.zip...")
with zipfile.ZipFile(rail_zip_path, "r") as z:
    z.extractall(RAIL_DIR)

print("Rail files:")
print(os.listdir(RAIL_DIR))

Extracting google_rail.zip...
Rail files:
['agency.txt', 'calendar.txt', 'calendar_dates.txt', 'directions.txt', 'feed_info.txt', 'routes.txt', 'route_stops.txt', 'shapes.txt', 'stops.txt', 'stop_times.txt', 'trips.txt']


## Load GTFS tables

- `routes`, `trips`, `stop_times` for trip building
- `calendar` for weekday service identification

In [17]:
routes = pd.read_csv(os.path.join(RAIL_DIR, "routes.txt"))
trips = pd.read_csv(os.path.join(RAIL_DIR, "trips.txt"))
stop_times = pd.read_csv(os.path.join(RAIL_DIR, "stop_times.txt"))
calendar = pd.read_csv(os.path.join(RAIL_DIR, "calendar.txt"))

print("routes:", routes.shape)
print("trips:", trips.shape)
print("stop_times:", stop_times.shape)
print("calendar:", calendar.shape)

routes: (13, 9)
trips: (1093, 7)
stop_times: (16323, 10)
calendar: (6, 10)


## Identify weekday service_id
Weekday = Monday–Friday service ON and Saturday/Sunday OFF.

In [18]:
weekday_services = calendar[
    (calendar["monday"] == 1) &
    (calendar["tuesday"] == 1) &
    (calendar["wednesday"] == 1) &
    (calendar["thursday"] == 1) &
    (calendar["friday"] == 1) &
    (calendar["saturday"] == 0) &
    (calendar["sunday"] == 0)
][["service_id", "start_date", "end_date"]]

weekday_services

,service_id,start_date,end_date
0,SID186346,20260215,20260405
4,SID185189,20260215,20260606
5,SID9185189,20260406,20260606


In [19]:
weekday_ids = weekday_services["service_id"].tolist()

service_counts = (
    trips[trips["service_id"].isin(weekday_ids)]
    .groupby("service_id")
    .size()
    .sort_values(ascending=False)
)

service_counts

service_id
SID185189     608
SID186346      24
SID9185189     24
dtype: int64

In [20]:
weekday_service_id = service_counts.index[0]
print("Using weekday_service_id:", weekday_service_id)

trips_weekday = trips[trips["service_id"] == weekday_service_id].copy()
print("trips_weekday:", trips_weekday.shape)

Using weekday_service_id: SID185189
trips_weekday: (608, 7)


## Build trip-level fact table (weekday)
Each row = one scheduled trip with route info, first departure time, stop count, and peak classification.

In [21]:
fact = trips_weekday.merge(routes, on="route_id", how="left")
print("fact after routes merge:", fact.shape)

fact after routes merge: (608, 15)


In [22]:
stop_count = (
    stop_times[stop_times["trip_id"].isin(fact["trip_id"])]
    .groupby("trip_id")
    .size()
    .reset_index(name="stop_count")
)

fact = fact.merge(stop_count, on="trip_id", how="left")
print("fact after stop_count:", fact.shape)

fact after stop_count: (608, 16)


In [23]:
first_departure = (
    stop_times[stop_times["trip_id"].isin(fact["trip_id"])]
    .sort_values(["trip_id", "stop_sequence"])
    .groupby("trip_id")
    .first()
    .reset_index()[["trip_id", "departure_time"]]
)

fact = fact.merge(first_departure, on="trip_id", how="left")
print("fact after departure_time:", fact.shape)

fact after departure_time: (608, 17)


In [24]:
def extract_hour(t):
    try:
        return int(str(t).split(":")[0])
    except:
        return None

def classify_peak(h):
    if h is None:
        return "Unknown"
    if 6 <= h <= 9:
        return "AM Peak"
    if 16 <= h <= 19:
        return "PM Peak"
    return "Off-Peak"

fact["departure_hour"] = fact["departure_time"].apply(extract_hour)
fact["service_period"] = fact["departure_hour"].apply(classify_peak)

fact[["trip_id","route_short_name","departure_time","departure_hour","service_period","stop_count"]].head()

,trip_id,route_short_name,departure_time,departure_hour,service_period,stop_count
0,CYN1052_20260201_SID185189,CYN,06:00:00,6,AM Peak,5
1,CYN1055_20260201_SID185189,CYN,06:46:00,6,AM Peak,5
2,CYN1056_20260201_SID185189,CYN,07:28:00,7,AM Peak,5
3,CYN1059_20260201_SID185189,CYN,08:10:00,8,AM Peak,5
4,CYN1060_20260201_SID185189,CYN,08:46:00,8,AM Peak,5


## Export Power BI tables (weekday-only)

In [25]:
fact_trips_weekday = fact[[
    "trip_id",
    "route_id",
    "route_short_name",
    "route_long_name",
    "departure_time",
    "departure_hour",
    "service_period",
    "stop_count",
    "service_id"
]].copy()

hourly_summary_weekday = (
    fact_trips_weekday
    .groupby("departure_hour")
    .size()
    .reset_index(name="trip_count")
    .sort_values("departure_hour")
)

route_summary_weekday = (
    fact_trips_weekday
    .assign(is_peak=fact_trips_weekday["service_period"].isin(["AM Peak", "PM Peak"]))
    .groupby("route_short_name")
    .agg(total_trips=("trip_id","count"), peak_trips=("is_peak","sum"))
    .reset_index()
)
route_summary_weekday["peak_ratio"] = route_summary_weekday["peak_trips"] / route_summary_weekday["total_trips"]

fact_trips_weekday.to_csv(os.path.join(EXPORT_DIR, "fact_trips_weekday.csv"), index=False)
hourly_summary_weekday.to_csv(os.path.join(EXPORT_DIR, "hourly_summary_weekday.csv"), index=False)
route_summary_weekday.to_csv(os.path.join(EXPORT_DIR, "route_summary_weekday.csv"), index=False)

print("Weekday total trips:", len(fact_trips_weekday))
print("Exports saved to:", EXPORT_DIR)

Weekday total trips: 608
Exports saved to: septa_weekday_dashboard\exports
